<a href="https://colab.research.google.com/github/Appolo-11/Neuro-science/blob/master/approx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.integrate import odeint

In [5]:
def odeintz(func, z0, t, **kwargs):
    """An odeint-like function for complex valued differential equations."""

    # Disallow Jacobian-related arguments.
    _unsupported_odeint_args = ['Dfun', 'col_deriv', 'ml', 'mu']
    bad_args = [arg for arg in kwargs if arg in _unsupported_odeint_args]
    if len(bad_args) > 0:
        raise ValueError("The odeint argument %r is not supported by "
                         "odeintz." % (bad_args[0],))

    # Make sure z0 is a numpy array of type np.complex128.
    z0 = np.array(z0, dtype=np.complex128, ndmin=1)

    def realfunc(x, t, *args):
        z = x.view(np.complex128)
        dzdt = func(z, t, *args)
        # func might return a python list, so convert its return
        # value to an array with type np.complex128, and then return
        # a np.float64 view of that array.
        return np.asarray(dzdt, dtype=np.complex128).view(np.float64)

    result = odeint(realfunc, z0.view(np.float64), t, **kwargs)

    if kwargs.get('full_output', False):
        z = result[0].view(np.complex128)
        infodict = result[1]
        return z, infodict
    else:
        z = result.view(np.complex128)
        return z

In [3]:
def circuit(tau, a, b, I, dist, sample_fr):
    def model(z, t, I):
        U = z[0]
        V = z[1]
        dU_dt = U - U**3/3 - V + I
        dV_dt = U + a - b*V
        dzdt = [dU_dt, dV_dt]
        return dzdt

    # initial condition
    z_0 = [0.000001, 0.000001]

    # number of time points

    # time points
    n = dist * sample_fr
    t = np.arange(0, dist,1/sample_fr)

    # step input
    I =np.sin(t)
    # store solution
    U = np.empty_like(t)
    V = np.empty_like(t)
    # record initial conditions
    U[0] = z_0[0]
    V[0] = z_0[1]
    # solve ODE
    for i in range(1, n):
        # span for next time step
        tspan = [t[i - 1], t[i]]
        # solve for next step
        z = odeint(model, z_0, tspan, args=(I[i],))
        # store solution for plotting
        U[i] = z[1][0]
        V[i] = z[1][1]
        # next initial condition
        z_0 = z[1]
    return t, U, V